In [2]:
# Importar librerías necesarias
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
import os
import datetime
import json

In [3]:
video_dir = 'C:\\Tesis\\TestErgo\\muestra'

In [4]:
def load_videos(video_dir):
    videos = []
    for filename in os.listdir(video_dir):
        if filename.endswith(".mp4"):
            videos.append(os.path.join(video_dir, filename))
    return videos

In [5]:
# Cargar videos
videos = load_videos(video_dir)

In [6]:
# Función para extraer frames de los videos
def extract_frames(video_path, interval=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % interval == 0:
            frames.append(frame)
        count += 1
    cap.release()
    return frames

In [7]:
# Obtener y mostrar los nombres de todos los videos
for video_path in videos:
    video_name = os.path.basename(video_path)
    print(video_name)

maquila014.mp4


In [8]:
# Inicializar MediaPipe para detección de puntos clave
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Función para calcular el ángulo entre tres puntos
def calculate_angle(a, b, c, d=None):
    if d is None:
        # Calcula el ángulo entre a, b y c
        a = np.array(a)  # Primer punto
        b = np.array(b)  # Segundo punto (vértice)
        c = np.array(c)  # Tercer punto

        radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
        angle = np.abs(radians * 180.0 / np.pi)

        if angle > 180.0:
            angle = 360.0 - angle

        return angle
    else:
        # Calcula el ángulo entre a, b, c y d
        a = np.array(a)  # Primer punto
        b = np.array(b)  # Segundo punto (vértice)
        c = np.array(c)  # Tercer punto
        d = np.array(d)  # Cuarto punto

        radians = np.arctan2(d[1] - c[1], d[0] - c[0]) - np.arctan2(b[1] - a[1], b[0] - a[0])
        angle = np.abs(radians * 180.0 / np.pi)

        if angle > 180.0:
            angle = 360.0 - angle

        return angle


In [9]:
# Función para detectar puntos clave usando MediaPipe
def detect_keypoints(video_path):
    cap = cv2.VideoCapture(video_path)
    pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
    keypoints_list = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        if results.pose_landmarks:
            keypoints = [(lm.x, lm.y, lm.z) for lm in results.pose_landmarks.landmark]
            keypoints_list.append(keypoints)
        else:
            keypoints_list.append([])  # Agregar una lista vacía si no se detectan keypoints

    cap.release()
    pose.close()
    return keypoints_list

In [10]:
# Función para analizar videos masivos y guardar resultados en un solo JSON
'''def analyze_videos_in_folder(video_folder, output_json_path):
    all_analysis_results = []
    
    for video_file in os.listdir(video_folder):
        if video_file.endswith(('.mp4', '.avi', '.mov')):
            video_path = os.path.join(video_folder, video_file)
            video_name = os.path.splitext(video_file)[0]
            
            keypoints_list = detect_keypoints(video_path)
            cap = cv2.VideoCapture(video_path)
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            cap.release()

            analysis_results = analyze_keypoints(keypoints_list, video_name, fps)
            all_analysis_results.extend(analysis_results)
            draw_keypoints_and_angles(video_path, analysis_results, output_folder, output_video_folder, json_folder)
    
    combined_json_path = os.path.join(json_folder, 'Resultado_analisis_combinados.json')
    with open(output_json_path, 'w') as json_file:
        json.dump(all_analysis_results, json_file, indent=4)

    print(f"Resultados del análisis exportados a: {output_json_path}")

# Ejemplo de uso:
#video_folder = 'C:\\Tesis\\TestErgo\\muestra'
video_folder = video_dir #'C:\\Tesis\\TestErgo\\VIDEOS MAQUILAS'
output_json_path = 'C:\\Tesis\\TestErgo\\analisis_videos.json'

analyze_videos_in_folder(video_folder, output_json_path)'''


'def analyze_videos_in_folder(video_folder, output_json_path):\n    all_analysis_results = []\n    \n    for video_file in os.listdir(video_folder):\n        if video_file.endswith((\'.mp4\', \'.avi\', \'.mov\')):\n            video_path = os.path.join(video_folder, video_file)\n            video_name = os.path.splitext(video_file)[0]\n            \n            keypoints_list = detect_keypoints(video_path)\n            cap = cv2.VideoCapture(video_path)\n            fps = int(cap.get(cv2.CAP_PROP_FPS))\n            cap.release()\n\n            analysis_results = analyze_keypoints(keypoints_list, video_name, fps)\n            all_analysis_results.extend(analysis_results)\n            draw_keypoints_and_angles(video_path, analysis_results, output_folder, output_video_folder, json_folder)\n    \n    combined_json_path = os.path.join(json_folder, \'Resultado_analisis_combinados.json\')\n    with open(output_json_path, \'w\') as json_file:\n        json.dump(all_analysis_results, json_file,

In [11]:
def analyze_keypoints(video_path):
    keypoints_list = detect_keypoints(video_path)
    fps = None
    analysis_results = []

    for frame_idx, keypoints in enumerate(keypoints_list):
        second = frame_idx // fps if fps else 0

        if not keypoints:
            analysis_results.append({
                'segundo': second,
                'frame': frame_idx,
                'video_name': None,
                'angulo_hombro_izquierdo': None,
                'angulo_del_hombro_derecho': None,
                'angulo_codo_izquierdo': None,
                'angulo_codo_derecho': None,
                'angulo_de_muneca_izquierda': None,
                'angulo_de_muneca_derecha': None,
                'angulo_mano_izquierdo': None,
                'angulo_mano_derecho': None,
                'posicion_hombro_izquierdo': None,
                'posicion_hombro_derecho': None,
                'posicion_codo_izquierdo': None,
                'posicion_codo_derecho': None,
                'posicion_muneca_izquierda': None,
                'posicion_muneca_derecha': None,
                'posicion_mano_izquierda': None,
                'posicion_mano_derecha': None,
                'keypoints': []
            })
            continue

        # Extraer puntos clave izquierdos
        left_shoulder = keypoints[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        left_elbow = keypoints[mp_pose.PoseLandmark.LEFT_ELBOW.value]
        left_wrist = keypoints[mp_pose.PoseLandmark.LEFT_WRIST.value]
        left_hand = keypoints[mp_pose.PoseLandmark.LEFT_INDEX.value]

        # Extraer puntos clave derechos
        right_shoulder = keypoints[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        right_elbow = keypoints[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
        right_wrist = keypoints[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        right_hand = keypoints[mp_pose.PoseLandmark.RIGHT_INDEX.value]

        # Calcular ángulos izquierdos
        left_shoulder_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        left_elbow_angle = calculate_angle(left_elbow, left_wrist, left_hand)
        left_wrist_angle = calculate_angle(left_elbow, left_wrist, left_hand)
        left_hand_angle = calculate_angle(left_elbow, left_wrist, left_hand)

        # Calcular ángulos derechos
        right_shoulder_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
        right_elbow_angle = calculate_angle(right_elbow, right_wrist, right_hand)
        right_wrist_angle = calculate_angle(right_elbow, right_wrist, right_hand)
        right_hand_angle = calculate_angle(right_elbow, right_wrist, right_hand)

        # Si no se ha establecido aún el FPS, se establece ahora
        if fps is None:
            cap = cv2.VideoCapture(video_path)
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            cap.release()

        analysis_results.append({
            'segundo': second,
            'frame': frame_idx,
            'analisis_video': video_path,
            'angulo_hombro_izquierdo': left_shoulder_angle,
            'angulo_del_hombro_derecho': right_shoulder_angle,
            'angulo_codo_izquierdo': left_elbow_angle,
            'angulo_codo_derecho': right_elbow_angle,
            'angulo_de_muneca_izquierda': left_wrist_angle,
            'angulo_de_muneca_derecha': right_wrist_angle,
            'angulo_mano_izquierdo': left_hand_angle,
            'angulo_mano_derecho': right_hand_angle,
            'posicion_hombro_izquierdo': left_shoulder,
            'posicion_hombro_derecho': right_shoulder,
            'posicion_codo_izquierdo': left_elbow,
            'posicion_codo_derecho': right_elbow,
            'posicion_muneca_izquierda': left_wrist,
            'posicion_muneca_derecha': right_wrist,
            'posicion_mano_izquierda': left_hand,
            'posicion_mano_derecha': right_hand,
            'keypoints': keypoints
        })

    return analysis_results, fps

In [12]:
def draw_keypoints_and_angles(video_path, output_folder, output_video_folder, json_folder):
    analysis_results, fps = analyze_keypoints(video_path)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error al abrir el video: {video_path}")
        return
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    video_frame_folder = os.path.join(output_folder, video_name)
    output_video_name = f"{video_name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"
    output_video_path = os.path.join(output_video_folder, output_video_name)
    json_filename = f"{video_name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    json_path = os.path.join(json_folder, json_filename)

    # Crear carpetas si no existen
    if not os.path.exists(video_frame_folder):
        os.makedirs(video_frame_folder)
    if not os.path.exists(output_video_folder):
        os.makedirs(output_video_folder)
    if not os.path.exists(json_folder):
        os.makedirs(json_folder)

    # Crear el video marcado
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    for i in range(len(analysis_results)):
        ret, frame = cap.read()
        if not ret:
            break

        keypoints = analysis_results[i]['keypoints']
        left_shoulder_angle = analysis_results[i]['angulo_hombro_izquierdo']
        right_shoulder_angle = analysis_results[i]['angulo_del_hombro_derecho']
        left_elbow_angle = analysis_results[i]['angulo_codo_izquierdo']
        right_elbow_angle = analysis_results[i]['angulo_codo_derecho']
        left_wrist_angle = analysis_results[i]['angulo_de_muneca_izquierda']
        right_wrist_angle = analysis_results[i]['angulo_de_muneca_derecha']
        left_hand_angle = analysis_results[i]['angulo_mano_izquierdo']
        right_hand_angle = analysis_results[i]['angulo_mano_derecho']

        # Dibujar puntos clave y conexiones con el color original
        if keypoints:
            for j, point in enumerate(keypoints):
                cv2.circle(frame, (int(point[0] * frame.shape[1]), int(point[1] * frame.shape[0])), 5, (0, 255, 0), -1)

                # Añadir texto con información relevante junto a cada punto
                if j == mp_pose.PoseLandmark.LEFT_SHOULDER.value:
                    cv2.putText(frame, f'Angulo del hombro izquierdo: {int(left_shoulder_angle) if left_shoulder_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.RIGHT_SHOULDER.value:
                    cv2.putText(frame, f'Angulo del hombro derecho: {int(right_shoulder_angle) if right_shoulder_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.LEFT_ELBOW.value:
                    cv2.putText(frame, f'Angulo del codo izquierdo: {int(left_elbow_angle) if left_elbow_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.RIGHT_ELBOW.value:
                    cv2.putText(frame, f'Angulo del codo derecho: {int(right_elbow_angle) if right_elbow_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.LEFT_WRIST.value:
                    cv2.putText(frame, f'Angulo de la muneca izquierda: {int(left_wrist_angle) if left_wrist_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.RIGHT_WRIST.value:
                    cv2.putText(frame, f'Angulo de la muneca derecha: {int(right_wrist_angle) if right_wrist_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.LEFT_INDEX.value:
                    cv2.putText(frame, f'Angulo de la mano izquierda: {int(left_hand_angle) if left_hand_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.RIGHT_INDEX.value:
                    cv2.putText(frame, f'Angulo de la mano derecha: {int(right_hand_angle) if right_hand_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

        # Guardar el frame en la carpeta de salida con el nombre del video seguido de su numeración
        output_frame_path = os.path.join(video_frame_folder, f'{video_name}_frame_{frame_count:04d}.png')
        cv2.imwrite(output_frame_path, frame)
        frame_count += 1

        # Guardar el frame en el video marcado
        out.write(frame)

    cap.release()
    out.release()
    
    # Guardar el archivo JSON con los resultados del análisis
    with open(json_path, 'w') as json_file:
        json.dump(analysis_results, json_file, indent=4)

    print(f"Frames exportados a: {video_frame_folder}")
    print(f"Video exportado a: {output_video_path}")
    print(f"JSON exportado a: {json_path}")
    return video_frame_folder, output_video_path, json_path


In [13]:
video_path = videos[0]
output_folder = 'C:\\Tesis\\TestErgo\\resultados'
output_video_folder = 'C:\\Tesis\\TestErgo\\videoMarcado'
json_folder = 'C:\\Tesis\\TestErgo\\archivos_json'

draw_keypoints_and_angles(video_path, output_folder, output_video_folder, json_folder)

c:\Tesis\TesisOCRA\TesisCheckListOCRA\python\venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Frames exportados a: C:\Tesis\TestErgo\resultados\maquila014
Video exportado a: C:\Tesis\TestErgo\videoMarcado\maquila014_20240725_102220.mp4
JSON exportado a: C:\Tesis\TestErgo\archivos_json\maquila014_20240725_102220.json


('C:\\Tesis\\TestErgo\\resultados\\maquila014',
 'C:\\Tesis\\TestErgo\\videoMarcado\\maquila014_20240725_102220.mp4',
 'C:\\Tesis\\TestErgo\\archivos_json\\maquila014_20240725_102220.json')

In [14]:
import os
import glob
import json
import cv2

def get_most_recent_file(folder_path, extension):
    """Encuentra el archivo más reciente con la extensión dada en la carpeta especificada."""
    list_of_files = glob.glob(os.path.join(folder_path, '*' + extension))
    if not list_of_files:
        raise FileNotFoundError(f"No se encontraron archivos con la extensión {extension} en {folder_path}.")
    return max(list_of_files, key=os.path.getmtime)

# Encuentra el archivo JSON y el video más recientes
json_folder = 'C:/Tesis/TestErgo/archivos_json'
video_folder = 'C:/Tesis/TestErgo/videoMarcado'

json_path = get_most_recent_file(json_folder, '.json')
video_path = get_most_recent_file(video_folder, '.mp4')

if json_path is None or video_path is None:
    raise FileNotFoundError("No se encontraron archivos JSON o de video en las carpetas especificadas.")

print(f"Archivo JSON más reciente: {json_path}")
print(f"Video más reciente: {video_path}")

# Función para calcular la duración total del turno en minutos
def get_duration_from_video(video_path):
    video = cv2.VideoCapture(video_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    duration_sec = total_frames / fps
    video.release()
    return duration_sec / 60  # Convertir a minutos

# Función para obtener el tiempo de trabajo no repetitivo en minutos
def get_non_repetitive_work_time(video_data):
    non_repetitive_time = sum(activity['duration'] for activity in video_data['activities'] if not activity['is_repetitive'])
    return non_repetitive_time / 60  # Convertir a minutos

# Función para identificar pausas y descansos
def get_breaks_and_rest_time(video_data):
    pauses = sum(break_time['duration'] for break_time in video_data['breaks'])
    rest = sum(rest_time['duration'] for rest_time in video_data['rests'])
    return pauses / 60, rest / 60  # Convertir a minutos

# Función para contar ciclos de trabajo
def get_number_of_cycles(video_data):
    return video_data['cycles_count']

# Extraer datos del video
duration_total = get_duration_from_video(video_path)
video_data = {
    'activities': [{'duration': 30, 'is_repetitive': False}],  # Ejemplo de datos, reemplazar con tu análisis real
    'breaks': [{'duration': 10}],  # Ejemplo de datos
    'rests': [{'duration': 30}],  # Ejemplo de datos
    'cycles_count': 120  # Ejemplo de datos
}

# Calcular TNTR y TNC
DT = duration_total
TNR = get_non_repetitive_work_time(video_data)
P, A = get_breaks_and_rest_time(video_data)
NC = get_number_of_cycles(video_data)

TNTR = DT - (TNR + P + A)
TNC = 60 * TNTR / NC

# Leer el archivo JSON existente
with open(json_path, 'r') as file:
    data = json.load(file)

# Verificar si el contenido es una lista
if isinstance(data, list):
    for item in data:
        item['Tiempo Neto de Trabajo Repetitivo'] = TNTR
        item['Tiempo Neto del Ciclo'] = TNC
else:
    data['Tiempo Neto de Trabajo Repetitivo'] = TNTR
    data['Tiempo Neto del Ciclo'] = TNC

# Guardar el JSON actualizado
with open(json_path, 'w') as file:
    json.dump(data, file, indent=4)

print("El JSON se ha actualizado con TNTR y TNC.")

Archivo JSON más reciente: C:/Tesis/TestErgo/archivos_json\maquila014_20240725_102220.json
Video más reciente: C:/Tesis/TestErgo/videoMarcado\maquila014_20240725_102220.mp4
El JSON se ha actualizado con TNTR y TNC.


Factor de Recuperación

In [15]:
import json

def calculate_fr(pauses, lunch_break_duration, shift_duration):
    """
    Cálculo de puntuación del FR basado en la Tabla 1
    """
    num_pauses = len(pauses)
    is_lunch_break_present = lunch_break_duration > 0

    if is_lunch_break_present and lunch_break_duration >= 8:
        if shift_duration >= 60:
            return 0  # Ideal
        elif num_pauses >= 4:
            return 2
    elif is_lunch_break_present and num_pauses >= 3:
        return 3
    elif num_pauses >= 2:
        return 4
    elif num_pauses == 1:
        return 6
    else:
        return 10  # Sin pausas adecuadas

# Definir la carpeta donde están los archivos JSON
json_folder = 'C:\\Tesis\\TestErgo\\archivos_json'
# Ruta del archivo JSON
json_path = get_most_recent_file(json_folder, '.json')

# Leer el archivo JSON
'''with open(json_path, 'r') as file:
    data = json.load(file)'''

# Leer el archivo JSON
with open(json_path, 'r') as file:
    data_list = json.load(file)

# Verificar si data_list es una lista y si no está vacía
if not isinstance(data_list, list) or len(data_list) == 0:
    raise ValueError("El archivo JSON no contiene una lista válida o está vacío.")

# Tomar el primer objeto de la lista
data = data_list[0]

# Extraer datos del JSON
pauses = [break_time for break_time in data.get('breaks', []) if break_time['duration'] >= 8]
lunch_break_duration = sum(rest_time['duration'] for rest_time in data.get('rests', []))
shift_duration = data.get('shift_duration', 480)  # Duración del turno en minutos, si no está en el JSON, usar valor por defecto

# Calcular el FR
fr = calculate_fr(pauses, lunch_break_duration, shift_duration)

# Añadir el cálculo al JSON
data['Factor de Recuperacion'] = fr

# Guardar el JSON actualizado
updated_json_path = json_path.replace('.json', 'FR.json')
with open(updated_json_path, 'w') as file:
    json.dump(data_list, file, indent=4)

print(f"El JSON se ha actualizado con el Factor de Recuperación y guardado en {updated_json_path}.")


El JSON se ha actualizado con el Factor de Recuperación y guardado en C:\Tesis\TestErgo\archivos_json\maquila014_20240725_102220FR.json.


Factor de frecuencia

In [16]:
import json

def calcular_ff(atd, ate):
    """
    Calcula el Factor de Frecuencia (FF) basado en las puntuaciones ATD y ATE.
    
    :param atd: Puntuación de Acciones Técnicas Dinámicas
    :param ate: Puntuación de Acciones Técnicas Estáticas
    :return: Puntuación del Factor de Frecuencia (FF)
    """
    return max(atd, ate)

# Definir la carpeta donde están los archivos JSON
json_folder = 'C:\\Tesis\\TestErgo\\archivos_json'
# Ruta del archivo JSON
json_path = get_most_recent_file(json_folder, '.json')

# Leer el archivo JSON
with open(json_path, 'r') as file:
    data_list = json.load(file)

# Verificar si data_list es una lista y si no está vacía
if not isinstance(data_list, list) or len(data_list) == 0:
    raise ValueError("El archivo JSON no contiene una lista válida o está vacío.")

# Tomar el primer objeto de la lista
data = data_list[0]

# Extraer puntuaciones del JSON
atd = data.get('AccionesTecnicasDinámicas', 0)
ate = data.get('AccionesTecnicasEstáticas', 0)

# Calcular el FF
ff = calcular_ff(atd, ate)

# Añadir el cálculo al JSON
data['Factor de Frecuencia'] = ff

# Guardar el JSON actualizado
updated_json_path = json_path.replace('.json', '__ff.json')
with open(updated_json_path, 'w') as file:
    json.dump(data_list, file, indent=4)

print(f"El JSON se ha actualizado con el Factor de Frecuencia y guardado en {updated_json_path}.")
print

El JSON se ha actualizado con el Factor de Frecuencia y guardado en C:\Tesis\TestErgo\archivos_json\maquila014_20240725_102220FR__ff.json.


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [17]:
import json

def calcular_ffz(acciones):
    """
    Calcula el Factor de Fuerza (FFz) basado en las acciones que requieren fuerza.
    
    :param acciones: Lista de acciones que requieren fuerza con sus detalles
    :return: Puntuación del Factor de Fuerza (FFz)
    """
    puntuacion_total = 0
    
    for accion in acciones:
        esfuerzo = accion['esfuerzo_cr10']
        duracion = accion['duracion']
        
        if esfuerzo in [3, 4]:  # Fuerza Moderada
            if duracion == "1/3 del tiempo":
                puntuacion_total += 2
            elif duracion == "50% del tiempo":
                puntuacion_total += 4
            elif duracion == "> 50% del tiempo":
                puntuacion_total += 6
            elif duracion == "Casi todo el tiempo":
                puntuacion_total += 8
                
        elif esfuerzo in [5, 6]:  # Fuerza Intensa
            if duracion == "2 seg. cada 10 min.":
                puntuacion_total += 4
            elif duracion == "1% del tiempo":
                puntuacion_total += 8
            elif duracion == "5% del tiempo":
                puntuacion_total += 16
            elif duracion == "> 10% del tiempo":
                puntuacion_total += 24
        
        elif esfuerzo >= 7:  # Fuerza Casi Máxima
            if duracion == "2 seg. cada 10 min.":
                puntuacion_total += 6
            elif duracion == "1% del tiempo":
                puntuacion_total += 12
            elif duracion == "5% del tiempo":
                puntuacion_total += 24
            elif duracion == "> 10% del tiempo":
                puntuacion_total += 32
    
    return puntuacion_total

# Definir la carpeta donde están los archivos JSON
json_folder = 'C:\\Tesis\\TestErgo\\archivos_json'
# Ruta del archivo JSON
json_path = get_most_recent_file(json_folder, '.json')

# Leer el archivo JSON
with open(json_path, 'r') as file:
    data_list = json.load(file)

# Verificar si data_list es una lista y si no está vacía
if not isinstance(data_list, list) or len(data_list) == 0:
    raise ValueError("El archivo JSON no contiene una lista válida o está vacío.")

# Tomar el primer objeto de la lista
data = data_list[0]

# Extraer acciones del JSON
acciones = data.get('acciones_fuerza', [])

# Calcular el FFz
ffz = calcular_ffz(acciones)

# Añadir el cálculo al JSON
data['Factor de Fuerza'] = ffz

# Guardar el JSON actualizado
updated_json_path = json_path.replace('.json', '_ffz.json')
with open(updated_json_path, 'w') as file:
    json.dump(data_list, file, indent=4)

print(f"El JSON se ha actualizado con el Factor de Fuerza y guardado en {updated_json_path}.")

El JSON se ha actualizado con el Factor de Fuerza y guardado en C:\Tesis\TestErgo\archivos_json\maquila014_20240725_102220FR__ff_ffz.json.


In [18]:
import json

def calculate_posture_movements(json_data):
    """
    Calcula el Factor de Posturas y Movimientos (FP) basado en el JSON proporcionado.
    """
    def get_max_value(*values):
        return max(values)

    # Obtener las puntuaciones individuales
    ph = json_data.get('PosturaHombro', 0)
    pc = json_data.get('PosturaCodo', 0)
    pm = json_data.get('PosturaMuñeca', 0)
    pa = json_data.get('PosturaMano', 0)
    pes = json_data.get('MovimientosEstereotipados', 0)

    # Calcular FP
    fp = get_max_value(ph, pc, pm, pa) + pes

    return fp

# Definir la carpeta donde están los archivos JSON
json_folder = 'C:\\Tesis\\TestErgo\\archivos_json'
# Ruta del archivo JSON
json_path = get_most_recent_file(json_folder, '.json')

# Leer el archivo JSON
with open(json_path, 'r') as file:
    data_list = json.load(file)

# Verificar si data_list es una lista y si no está vacía
if not isinstance(data_list, list) or len(data_list) == 0:
    raise ValueError("El archivo JSON no contiene una lista válida o está vacío.")

# Tomar el primer objeto de la lista
data = data_list[0]

# Calcular el FP
fp = calculate_posture_movements(data)

# Añadir el cálculo al JSON
data['Factor de Posturas y Movimientos'] = fp

# Guardar el JSON actualizado
updated_json_path = json_path.replace('.json', '_fp.json')
with open(updated_json_path, 'w') as file:
    json.dump(data_list, file, indent=4)

print(f"El JSON se ha actualizado con el Factor de Posturas y Movimientos y guardado en {updated_json_path}.")


El JSON se ha actualizado con el Factor de Posturas y Movimientos y guardado en C:\Tesis\TestErgo\archivos_json\maquila014_20240725_102220FR__ff_ffz_fp.json.


In [19]:
import json

def calcular_md(tntr):
    """
    Calcula el Multiplicador de Duración (MD) basado en el Tiempo Neto de Trabajo Repetitivo (TNTR).
    
    :param tntr: Tiempo Neto de Trabajo Repetitivo en minutos
    :return: Puntuación del Multiplicador de Duración (MD)
    """
    if tntr <= 60:
        return 0.5
    elif 60 < tntr <= 120:
        return 0.5
    elif 120 < tntr <= 180:
        return 0.65
    elif 180 < tntr <= 240:
        return 0.75
    elif 240 < tntr <= 300:
        return 0.85
    elif 300 < tntr <= 360:
        return 0.925
    elif 360 < tntr <= 420:
        return 0.95
    elif 420 < tntr <= 480:
        return 1
    elif 480 < tntr <= 539:
        return 1.2
    elif 539 < tntr <= 599:
        return 1.5
    elif 599 < tntr <= 659:
        return 2
    elif 659 < tntr <= 719:
        return 2.8
    elif tntr >= 720:
        return 4
    else:  # Para análisis multitarea
        if tntr <= 1.87:
            return 0.01
        elif 1.87 < tntr <= 3.75:
            return 0.02
        elif 3.75 < tntr <= 7.5:
            return 0.05
        elif 7.5 < tntr <= 15:
            return 0.1
        elif 15 < tntr <= 30:
            return 0.2
        elif 30 < tntr <= 59:
            return 0.35

# Definir la carpeta donde están los archivos JSON
json_folder = 'C:\\Tesis\\TestErgo\\archivos_json'
# Ruta del archivo JSON
json_path = get_most_recent_file(json_folder, '.json')

# Leer el archivo JSON
with open(json_path, 'r') as file:
    data_list = json.load(file)

# Verificar si data_list es una lista y si no está vacía
if not isinstance(data_list, list) or len(data_list) == 0:
    raise ValueError("El archivo JSON no contiene una lista válida o está vacío.")

# Tomar el primer objeto de la lista
data = data_list[0]

# Extraer TNTR del JSON
tntr = data.get('TNTR', 480)  # Usar 480 minutos por defecto si no está presente

# Calcular el MD
md = calcular_md(tntr)

# Añadir el cálculo al JSON
data['Multiplicador de Duracion'] = md

# Guardar el JSON actualizado
updated_json_path = json_path.replace('.json', '_md.json')
with open(updated_json_path, 'w') as file:
    json.dump(data_list, file, indent=4)

print(f"El JSON se ha actualizado con el Multiplicador de Duración y guardado en {updated_json_path}.")

El JSON se ha actualizado con el Multiplicador de Duración y guardado en C:\Tesis\TestErgo\archivos_json\maquila014_20240725_102220FR__ff_ffz_fp_md.json.


In [21]:
import json
import pandas as pd
from datetime import datetime
import os

def calcular_md(tntr):
    """
    Calcula el Multiplicador de Duración (MD) basado en el Tiempo Neto de Trabajo Repetitivo (TNTR).
    """
    if tntr <= 60:
        return 0.5
    elif 60 < tntr <= 120:
        return 0.5
    elif 120 < tntr <= 180:
        return 0.65
    elif 180 < tntr <= 240:
        return 0.75
    elif 240 < tntr <= 300:
        return 0.85
    elif 300 < tntr <= 360:
        return 0.925
    elif 360 < tntr <= 420:
        return 0.95
    elif 420 < tntr <= 480:
        return 1
    elif 480 < tntr <= 539:
        return 1.2
    elif 539 < tntr <= 599:
        return 1.5
    elif 599 < tntr <= 659:
        return 2
    elif 659 < tntr <= 719:
        return 2.8
    elif tntr >= 720:
        return 4
    else:  # Para análisis multitarea
        if tntr <= 1.87:
            return 0.01
        elif 1.87 < tntr <= 3.75:
            return 0.02
        elif 3.75 < tntr <= 7.5:
            return 0.05
        elif 7.5 < tntr <= 15:
            return 0.1
        elif 15 < tntr <= 30:
            return 0.2
        elif 30 < tntr <= 59:
            return 0.35

def calcular_checklist_ocra(data):
    """
    Calcula el Índice Check List OCRA (ICKL) basado en los factores y el multiplicador de duración (MD).
    """
    video_name = data.get('video_name')
    tntr = data.get('TNTR', 480)  # Tiempo Neto de Trabajo Repetitivo en minutos

    # Calcular el Multiplicador de Duración
    md = calcular_md(tntr)

    # Obtener los factores del JSON
    fr = data.get('Factor de Recuperacion', 0)
    ff = data.get('Factor de Frecuencia', 0)
    ffz = data.get('Factor de Fuerza', 0)
    fp = data.get('Factor de Posturas y Movimientos', 0)

    # Calcular el Índice Check List OCRA
    ickl = (fr + ff + ffz + fp) * md

    # Resumen de ángulos y puntos
    resumen = {
        'Promedio Hombro': data.get('Promedio Hombro', 0),
        'Máximo Hombro': data.get('Máximo Hombro', 0),
        'Mínimo Hombro': data.get('Mínimo Hombro', 0),
        'Estado Promedio Hombro': data.get('Estado Promedio Hombro', 'No disponible'),
        'Estado Máximo Hombro': data.get('Estado Máximo Hombro', 'No disponible'),
        'Estado Mínimo Hombro': data.get('Estado Mínimo Hombro', 'No disponible'),
        'Promedio Codo': data.get('Promedio Codo', 0),
        'Máximo Codo': data.get('Máximo Codo', 0),
        'Mínimo Codo': data.get('Mínimo Codo', 0),
        'Estado Promedio Codo': data.get('Estado Promedio Codo', 'No disponible'),
        'Estado Máximo Codo': data.get('Estado Máximo Codo', 'No disponible'),
        'Estado Mínimo Codo': data.get('Estado Mínimo Codo', 'No disponible'),
        'Promedio Muñeca': data.get('Promedio Muñeca', 0),
        'Máximo Muñeca': data.get('Máximo Muñeca', 0),
        'Mínimo Muñeca': data.get('Mínimo Muñeca', 0),
        'Estado Promedio Muñeca': data.get('Estado Promedio Muñeca', 'No disponible'),
        'Estado Máximo Muñeca': data.get('Estado Máximo Muñeca', 'No disponible'),
        'Estado Mínimo Muñeca': data.get('Estado Mínimo Muñeca', 'No disponible'),
        'Promedio Dedos': data.get('Promedio Dedos', 0),
        'Máximo Dedos': data.get('Máximo Dedos', 0),
        'Mínimo Dedos': data.get('Mínimo Dedos', 0),
        'Estado Promedio Dedos': data.get('Estado Promedio Dedos', 'No disponible'),
        'Estado Máximo Dedos': data.get('Estado Máximo Dedos', 'No disponible'),
        'Estado Mínimo Dedos': data.get('Estado Mínimo Dedos', 'No disponible')
    }

    result = {
        'Video': video_name,
        'Factor de Recuperacion': fr,
        'Factor de Frecuencia': ff,
        'Factor de Fuerza': ffz,
        'Factor de Posturas y Movimientos': fp,
        'Multiplicador de Duración': md,
        'Índice Check List OCRA (ICKL)': ickl,
        **resumen
    }

    return result

def guardar_resultado_excel(result):
    """
    Guarda los resultados del análisis en un archivo Excel.
    """
    # Construir el nombre del archivo
    now = datetime.now().strftime('%Y%m%d_%H%M%S')
    file_name = f'AnalisisCheckListOCRA_{result["Video"]}_{now}.xlsx'

    # Definir la ruta completa
    folder_path = 'C:\\Tesis\\TesisOCRA\\TesisCheckListOCRA\\static\\resultados'
    os.makedirs(folder_path, exist_ok=True)
    file_path = os.path.join(folder_path, file_name)

    # Convertir el resultado a un DataFrame de pandas
    df = pd.DataFrame([result])

    # Guardar el DataFrame a un archivo Excel
    df.to_excel(file_path, index=False, engine='openpyxl')

    print(f'El archivo Excel se ha guardado en: {file_path}')

# Definir la carpeta donde están los archivos JSON
json_folder = 'C:\\Tesis\\TestErgo\\archivos_json'
# Ruta del archivo JSON
json_path = get_most_recent_file(json_folder, '.json')

# Leer el archivo JSON
with open(json_path, 'r') as file:
    data_list = json.load(file)

# Verificar si data_list es una lista y si no está vacía
if not isinstance(data_list, list) or len(data_list) == 0:
    raise ValueError("El archivo JSON no contiene una lista válida o está vacío.")

# Tomar el primer objeto de la lista
data = data_list[0]

# Calcular el Check List OCRA
result = calcular_checklist_ocra(data)

# Guardar el resultado en un archivo Excel
guardar_resultado_excel(result)


UnboundLocalError: cannot access local variable 'video_name' where it is not associated with a value